In [2]:
# Sampling process with Jump : Black-Scholes-Merton Option Pricing with Jump

import numpy as np
from scipy import stats

# BSM closed-form solution (for call)
def bsm_call_value(S0, K, T, r, sigma):
    d1= (np.log(S0/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2= (np.log(S0/K)+(r-0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    call_value = (S0*stats.norm.cdf(d1,0.0,1.0) - K*np.exp(-r*T)*stats.norm.cdf(d2,0.0,1.0))
    return call_value


# BSMOPM params.
S_0= 100.
K= 105.
T= 2.
r= 0.05
sigma= 0.2
avg_jump_num = 2.0
avg_jump_logsize = 0.0
vol_jump_logsize = 0.2

# Risk-neutral valuation 보정위한 계수 eta
eta = np.exp(avg_jump_logsize + vol_jump_logsize**2/2)-1

# Monte Carlo Simulation
N = int(1e5)         # Number of sample paths
W_T = np.random.normal(loc=0.0, scale=np.sqrt(T), size=N)
jump_num = np.random.poisson(lam=avg_jump_num*T, size=N)
jump_size = np.ones(N)

for i in range(N):
    if(jump_num[i]>0):
        jump_size[i] = np.prod( np.exp( np.random.normal(loc=avg_jump_logsize, scale=vol_jump_logsize, size=jump_num[i])))
        
S_T = S_0*np.exp((r-avg_jump_num*eta-0.5*sigma*sigma)*T + sigma*W_T)*jump_size # RN 적용시 보정하기 위한 term w
C_0 = np.exp(-r*T)*(S_T-K)*(S_T > K)

print('Estimated Call Price (95% C.I.) = ',np.mean(C_0),' +- ',1.96*np.std(C_0)/np.sqrt(N))
print('BSM Call Price = ', bsm_call_value(S_0,K,T,r,sigma))

Estimated Call Price (95% C.I.) =  21.38026513421025  +-  0.2624285371280228
BSM Call Price =  13.639615096767713


In [3]:
# CDS_spread_flat_intensity with RN

import numpy as np

np.random.seed(0)

M = int(1e6)   # Number of simulation paths

default_intensity = 0.005 # 대략 200년에 한번 default 날 가능성 
loss_given_default = 0.6  # 1-회수율(0.4)

T = 5.0  # 5년만기
r = 0.02 # 이자율

D_0 = 0.0 # default leg
P_0 = 0.0 # premium leg

for idx in range(M):
    default_time = np.random.exponential(1.0/default_intensity) # calualate default_time
    
    if(default_time < T): # CDS 만기 이전에 default 발생시
        D_0 = D_0 + np.exp(-r*default_time)*loss_given_default
        
    t_m = 0.0
    while(t_m < T): # 만기이전
        t_m = t_m + 0.25 # 3개월마다 한번씩 quaterly payment Cm (=0.25) 누적합
        if(t_m < default_time):
            P_0 = P_0 + np.exp(-r*t_m)*0.25
            
spread = D_0 / P_0 * 10000    # fair spread in basis point : 1bps = 0.0001 ??
print("Fair spread = ",spread, " (bps)")

Fair spread =  30.005559556052  (bps)
